In [1]:
import pandas as pd

df = pd.read_csv('../data/processed/headlines_cl_sent.csv')

In [2]:
import ast
df['clean_hl_words'] = df['clean_hl_words'].apply(lambda row: ast.literal_eval(row))
# turn words into sets so we can compute intersection
df['clean_hl_words'] = df['clean_hl_words'].apply(lambda row: set([w.lower() for w in row]))
df['hl_words'] = df['headline_no_site'].apply(lambda row: set([w.lower() for w in row.split()]))

In [52]:
from datetime import datetime
from datetime import timedelta
import time

In [39]:
def calc_publishing_time(scrape_date, duration):
    date = 0
    if duration[1] in ['week', 'weeks', 'weken']:
        date = datetime(scrape_date) - timedelta(weeks=int(duration[0]))
    elif duration[1] in ['days', 'day', 'dagen']:
        date = datetime(scrape_date) - timedelta(days=int(duration[0]))
    elif duration[1] in ['hour', 'hours', 'uur']:
        date = datetime(scrape_date) - timedelta(hours=int(duration[0]))
    return date
    

In [44]:
int(df['time'].str.split(' ')[0][0])

4

In [56]:
#calc_publishing_time(df['scrape_date'][0], df['time'].str.split(' ')[0])

In [18]:
df['time'].str.split(' ')[0]

0        [4, dagen, geleden]
1         [1, week, geleden]
2        [5, dagen, geleden]
3         [17, uur, geleden]
4         [1, week, geleden]
                ...         
47549    [3, weken, geleden]
47550     [1, week, geleden]
47551    [3, weken, geleden]
47552     [1, week, geleden]
47553      [1, uur, geleden]
Name: time, Length: 47554, dtype: object

In [104]:
df.shape

(47554, 24)

In [115]:
#Two lists  of words that are used when a man or woman is present, based on Danielle Sucher's https://github.com/DanielleSucher/Jailbreak-the-Patriarchy
# male_words=set(['guy','spokesman','chairman',"men's",'men','him',"he's",'his','boy','boyfriend','boyfriends','boys','brother','brothers','dad','dads','dude','father','fathers','fiance','gentleman','gentlemen','god','grandfather','grandpa','grandson','groom','he','himself','husband','husbands','king','male','man','mr','nephew','nephews','priest','prince','son','sons','uncle','uncles','waiter','widower','widowers'])
female_words=set(['heroine','spokeswoman','chairwoman',"women's",'actress','women',"she's",'her','aunt',
                  'aunts','bride','daughter','daughters','female','fiancee','girl','girlfriend','girlfriends',
                  'girls','goddess','granddaughter','grandma','grandmother','herself','ladies','lady','lady',
                  'mom','moms','mother','mothers','mrs','ms','niece','nieces','priestess','princess',
                  'she','sister','sisters','waitress','widow','widows','wife','wives','woman', 'beautiful', 
                  'beauty', 'she', 'housewife', 'queen', 'gender', 'pregnant', 'queen', 'madam', 'mum' ])

bias_words = set(['fair', 'weight', 'beauty', 'frail', 'weak', 'modest', 'virgin', 
                  'sex', 'feminine', 'sensitive', 'emotional', 'gentle', 'soft', 
                  'pretty', 'bitch', 'sexual', 'harassment', 'shopping','shop',
                  'sale', 'cook', 'cooking', 'sexual', 'baby', 'child', 'breast', 'breasts'])

male_words = set(['hero', 'man',  'men', 'his', 'him', 'he', 'husband', 'father', 'male', 'son', 'god',
                  'prince', 'king', 'mr', 'sir', 'brother', 'grandfather', 'uncle', 'nephew'])

empowerement_words = set(['chairperson', 'leader', 'chairwoman', 'minister', 'power', 'authority', 'queen', 'manager'])

In [121]:
# def gender_the_sentence(sentence_words):
#     mw_length=len(male_words.intersection(sentence_words))
#     fw_length=len(female_words.intersection(sentence_words))

#     if mw_length>0 and fw_length==0:
#         gender='male'
#     elif mw_length==0 and fw_length>0: 
#         gender='female'
#     elif mw_length>0 and fw_length>0: 
#         gender='both'
#     else:
#         gender='none'
#     return gender

In [39]:
def gender_the_sentence(sentence_words, type_words):
#     mw_length=len(male_words.intersection(sentence_words))
    fw_length=len(type_words.intersection(sentence_words))

#     if mw_length>0 and fw_length==0:
#         gender='male'
    if fw_length>0: 
        gender=1
    else:
        gender=0
    return gender

In [81]:
def quantify_bias(sentence_words, type_words):
#     mw_length=len(male_words.intersection(sentence_words))
    fw_length=len(type_words.intersection(sentence_words))
    bias_length = len(bias_words.intersection(sentence_words))
#     if bias_length>0:
#         print(sentence_words)
#         print(bias_length)

#     if mw_length>0 and fw_length==0:
#         gender='male'
    if fw_length>0 and type_words == 'female_words': 
        bias=fw_length + bias_length
    elif fw_length>0 :
        bias = fw_length
    else:
        bias=0
    return bias

In [6]:
df['clean_hl_words']

0        {william, waarheid, moet, s, uit, diana, onder...
1        {k, die, ooievaar, plaaslike, onder, doenig, b...
2                       {artist, coast, dream, west, live}
3                            {beckon, cillier, apple, big}
4        {alderman, bazier, honours, councillor, title,...
                               ...                        
47549              {hospital, life, crash, fight, maghull}
47550    {ex, slap, treat, fiancee, finger, break, canc...
47551    {disabled, thug, ear, bloodstained, bite, blad...
47552    {heartbreaking, dad, share, wish, terminal, ca...
47553    {worker, food, order, mconald, share, wrong, f...
Name: clean_hl_words, Length: 47554, dtype: object

In [116]:
# create bias columns
df['gendered'] = df['hl_words'].apply(lambda row: gender_the_sentence(row, female_words))
df['bias'] = df['hl_words'].apply(lambda row: quantify_bias(row, female_words))
df['empowerement'] = df['hl_words'].apply(lambda row:quantify_bias(row, empowerement_words) )
df['male_words'] = df['hl_words'].apply(lambda row:quantify_bias(row, male_words) )

In [117]:
df[df['empowerement'] > 1]['headline']

29521    Bolivia’s interim leader says she will ‘fight’...
34409    Queen Tiye and Amenhotep III shared power to r...
40879    New Zealand’s Prime Minister May Be the Most E...
Name: headline, dtype: object

In [120]:
df[df['bias']==0]

,Unnamed: 0,url,headline,subtitle,time,scrape_date,site,monthly_visits,country,country_of_pub,...,clean_hl_words,compound_s,positive_s,negative_s,neutral_s,hl_words,gendered,bias,empowerement,male_words
0,0,https://www.netwerk24.com/Vermaak/Celebs/waarh...,"Waarheid moet uit oor Diana-onderhoud, s� Will...",WilliamNetwerk244 dagen geleden,4 dagen geleden,23/11/2020,Netwerk24.com,14000000,South Africa,South Africa,...,"{william, waarheid, moet, s, uit, diana, onder...",0.0000,0.000,0.000,1.000,"{waarheid, s�, moet, uit, diana-onderhoud, wil...",0,0,0,0
1,1,https://www.netwerk24.com/huisgenoot/Bekendes/...,KYK: Die ooievaar is doenig onder plaaslike be...,Hier is van die oulikste foto's wat plaaslike ...,1 week geleden,23/11/2020,Netwerk24.com,14000000,South Africa,South Africa,...,"{k, die, ooievaar, plaaslike, onder, doenig, b...",-0.5994,0.000,0.358,0.642,"{die, ooievaar, plaaslike, onder, k:, doenig, ...",0,0,0,0
3,3,https://www.netwerk24.com/ZA/Distrikspos/Nuus/...,Big Apple beckons for CilliersNetwerk24,An aspiring artist from Somerset West will put...,17 uur geleden,23/11/2020,Netwerk24.com,14000000,South Africa,South Africa,...,"{beckon, cillier, apple, big}",0.0000,0.000,0.000,1.000,"{apple, for, beckons, big, cilliers}",0,0,0,0
4,4,https://www.netwerk24.com/ZA/Distrikspos/Nuus/...,City honours PR councillor Bazier with Alderma...,A local proportional representative (PR) counc...,1 week geleden,23/11/2020,Netwerk24.com,14000000,South Africa,South Africa,...,"{alderman, bazier, honours, councillor, title,...",0.4939,0.314,0.000,0.686,"{with, alderman, bazier, honours, councillor, ...",0,0,0,0
6,6,https://www.netwerk24.com/ZA/UD-Express/someth...,'Something is terribly wrong in society'Netwerk24,WOMEN in South Africa are in danger. There is ...,4 dagen geleden,23/11/2020,Netwerk24.com,14000000,South Africa,South Africa,...,"{wrong, omething, society, terribly}",-0.7717,0.000,0.626,0.374,"{society', terribly, 'omething, wrong, is, in}",0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47513,47513,https://www.liverpoolecho.co.uk/news/liverpool...,"Friends, colleagues and loved ones pay tribute...",Tributes poured in following the death of a 56...,2 dagen geleden,22/11/2020,liverpoolecho.co.uk,11300000,UK,UK,...,"{love, dignified, pay, tribute, paula, one, le...",0.9217,0.591,0.061,0.348,"{ones, friends, to, and, pay, tribute, 'brave,...",0,0,0,0
47515,47515,https://www.liverpoolecho.co.uk/news/liverpool...,Predatory paedophile ruined life of ninth vict...,Phillip Taylor groomed the parents of girls - ...,2 dagen geleden,22/11/2020,liverpoolecho.co.uk,11300000,UK,UK,...,"{victim, ninth, ruin, abuse, paedophile, life,...",-0.9136,0.000,0.679,0.321,"{victim, with, ninth, ruined, abuse, paedophil...",0,0,0,0
47519,47519,https://www.liverpoolecho.co.uk/news/tv/kate-g...,Kate Garraway feels 'sick' at thought of Chris...,"Kate Garraway has said she feels ""physically s...",2 dagen geleden,22/11/2020,liverpoolecho.co.uk,11300000,UK,UK,...,"{garraway, christmas, kate, derek, feel, thoug...",-0.5106,0.000,0.268,0.732,"{garraway, at, 'sick', feels, christmas, witho...",0,0,0,0
47539,47539,https://www.liverpoolecho.co.uk/news/tv/amanda...,Amanda Holden's baby faced Blind Date appearan...,Amanda never made it down the aisle with her u...,2 dagen geleden,22/11/2020,liverpoolecho.co.uk,11300000,UK,UK,...,"{baby, appearance, amanda, blind, holden, face...",-0.4019,0.000,0.252,0.748,"{baby, 1991, faced, appearance, blind, amanda,...",0,0,0,0


In [121]:
df[df['male_words']>2]['headline'][35226]

'Pakistani Man Wants Fourth Wedding and His Three Wives are Helping Him Find a New BrideNews18'